## TOC:
* [Near text filer](#near-text-filer)

In [1]:
import weaviate
from datetime import datetime, timezone
from termcolor import colored
from typing import Optional
import re


In [2]:
def pprint_response(response: dict, description_size_limit: Optional[int] = 1_000) -> None:

    response = response["data"]["Get"]

    for class_name in response:
        print(colored(class_name.upper(), "red", "on_white", attrs=["bold"]))

        for idx, resp in enumerate(response[class_name]):
            print(f"Response # {idx}")
            for key, value in resp.items():
                if key in {"description", "short_description"} and description_size_limit:
                    value = value[:description_size_limit]
                print(f"{colored(key, 'magenta', 'on_grey')}: {value}")
            print("=" * 120)


In [3]:
def get_class_count(client: weaviate.Client, class_name: str) -> int:
    response = client.query.aggregate(class_name).with_meta_count().do()
    return response["data"]["Aggregate"][class_name][0]["meta"]["count"]


In [4]:
client = weaviate.Client("http://localhost:8080")


In [5]:
articles_count = get_class_count(client, "Article")
authors_count = get_class_count(client, "Author")
print(f"Articles: {articles_count}; Authors: {authors_count}\n")


Articles: 593; Authors: 187



# 1. Requests with filters that are supported by builtin methods 

## 1.1. Near text filer (vector search) <a class="anchor" id="near-text-filer"></a>

In [6]:
class_name = "Article"
properties = [
    "title",
    # "short_description",
    "_additional {certainty}",
]

nearText = {
    "concepts": ["fashion"],
    "certainty": 0.5,
}

response = client.query.get(class_name, properties).with_near_text(nearText).with_limit(5).do()

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.6303598}
title: Nadja Swarovski: The luxury sector has been ‘incredibly resilient’ amid the pandemic
Response # 1
_additional: {'certainty': 0.6030835}
title: The world’s most powerful women are …
Response # 2
_additional: {'certainty': 0.57555836}
title: Luxury Motorbike Sales Accelerate in India
Response # 3
_additional: {'certainty': 0.56418556}
title: Star Wars meets Uniqlo as Fast Retailing, Disney expand product tie-up
Response # 4
_additional: {'certainty': 0.5591956}
title: Amazon's PillPack is battling with CVS and Walgreens over getting patient prescriptions


In [7]:
nearText = {
    "concepts": ["fashion"],
    "certainty": 0.5,
    "moveAwayFrom": {
        "concepts": ["financial"],
        "force": 0.8,
    },
}

response = client.query.get(class_name, properties).with_near_text(nearText).with_limit(5).do()

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.628858}
title: Nadja Swarovski: The luxury sector has been ‘incredibly resilient’ amid the pandemic
Response # 1
_additional: {'certainty': 0.586373}
title: Luxury Motorbike Sales Accelerate in India
Response # 2
_additional: {'certainty': 0.57763237}
title: The world’s most powerful women are …
Response # 3
_additional: {'certainty': 0.57247543}
title: Star Wars meets Uniqlo as Fast Retailing, Disney expand product tie-up
Response # 4
_additional: {'certainty': 0.5706278}
title: Amazon's PillPack is battling with CVS and Walgreens over getting patient prescriptions


In [8]:
neartext = {
    "concepts": ["fashion"],
    "certainty": 0.5,
    "moveawayfrom": {
        "concepts": ["financial"],
        "force": 0.8,
    },
    "moveto": {
        "concepts": ["couturier"],
        "force": 0.85,
    },
}
response = client.query.get(class_name, properties).with_near_text(nearText).with_limit(5).do()

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.628858}
title: Nadja Swarovski: The luxury sector has been ‘incredibly resilient’ amid the pandemic
Response # 1
_additional: {'certainty': 0.586373}
title: Luxury Motorbike Sales Accelerate in India
Response # 2
_additional: {'certainty': 0.57763237}
title: The world’s most powerful women are …
Response # 3
_additional: {'certainty': 0.57247543}
title: Star Wars meets Uniqlo as Fast Retailing, Disney expand product tie-up
Response # 4
_additional: {'certainty': 0.5706278}
title: Amazon's PillPack is battling with CVS and Walgreens over getting patient prescriptions


## 1.2. Filter by date

In [9]:
date = datetime(year=2020, month=1, day=1, tzinfo=timezone.utc).isoformat()
print(date)


2020-01-01T00:00:00+00:00


In [10]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "short_description",
            # "description",
            "published_at",
            "_additional {certainty}",
        ],
    )
    .with_near_text(nearText)
    .with_where(
        {
            "operator": "LessThan",
            "path": ["published_at"],
            # valueDate: The date (ISO 8601 timestamp, formatted as RFC3339) value that the Path’s last property name should be compared to.
            "valueDate": date,
        }
    )
    .with_limit(3)
    .do()
)

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.71583223}
published_at: 2017-09-20T10:05:17Z
short_description: Cybersecurity, health of the global economy, energy price shock and terrorist attacks are some of the top risks that concern businesses and may threaten their ability to operate, according to results from a new survey by the World Economic Forum (WEF) published Wednesday.The survey, conducted annually by the WEF's strategic partners Marsh & McLennan Companies and Zurich Insurance Group, highlighted the following ten risks that businesses are presently concerned about:While concerns about the overall health of the global economy continue to dominate the list, the report found concerns around cybersecurity risks rising in importance."Business leaders in many of world's largest economies now rank cyber as their top risk," John Drzik, president of Marsh Global Risk & Digital, said in a press statement. "Companies need to rigorously analyze how these threats could impact their o

## 1.3. Filter by keywords

In [11]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

class_name = "Article"
properties = [
    "title",
    "keywords",
    "_additional {certainty}",
]

response = (
    client.query.get(class_name, properties)
    .with_near_text(nearText)
    .with_where(
        {
            "operator": "Equal",
            "path": ["keywords"],
            "valueText": ["bonds"],  # there are no articles with these keywords at the same time
        }
    )
    .with_limit(2)
    .do()
)

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.6724188}
keywords: ['cnbc', 'Articles', 'World Markets', 'Bonds', 'Markets', 'Corporate bonds', 'Investment strategy', 'Corporate Debt', 'Investing', 'Credit and Debt', 'Squawk on the Street', 'Market Outlook', 'Europe Economy', 'source:tagname:CNBC US Source']
title: Lasry: Europe could face Japan-style deflation era
Response # 1
_additional: {'certainty': 0.60360354}
keywords: ['cnbc', 'Articles', 'Commodity markets', 'Currency markets', 'Bonds', 'Barclays PLC', 'HSBC Holdings PLC', 'Meta Platforms Inc', 'Microsoft Corp', 'Alphabet Class A', 'International Business Machines Corp', 'Howmet Aerospace Inc', 'Citigroup Inc', 'Goldman Sachs Group Inc', 'Currencies', 'Futures & Commodities', 'DOW 30', 'Markets', 'stocks', 'Stock Blog', 'source:tagname:CNBC US Source']
title: The Stocks Analysts Are Talking About


In [12]:
response = (
    client.query.get(class_name, properties)
    .with_near_text(nearText)
    .with_where(
        {
            "operator": "Equal",
            "path": ["keywords"],
            "valueText": ["fast money"],  # there are no articles with these keywords at the same time
        }
    )
    .with_limit(2)
    .do()
)

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.65679705}
keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money Halftime Report', 'source:tagname:CNBC US Source']
title: EU, Iran to Impact Crude Oil Prices, Experts Say
Response # 1
_additional: {'certainty': 0.6142397}
keywords: ['cnbc', 'Articles', 'S&P 500 Index', 'Bank of America Corp', 'BHP Group Ltd', 'Freeport-McMoRan Inc', 'HP Inc', 'Intel Corp', 'Oracle Corp', 'US Bancorp', 'CBOE Volatility Index', 'United States Steel Corp', 'Industrial Select Sector SPDR Fund', "McDonald's Corp", 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
title: Sellers Worried about Europe, or Playing Range Game?


In [13]:
response = (
    client.query.get(class_name, properties)
    .with_near_text(nearText)
    .with_where(
        {
            "operator": "Equal",
            "path": ["keywords"],
            "valueText": ["bonds", "fast money"],  # there are no articles with these keywords at the same time
        }
    )
    .with_limit(5)
    .do()
)

pprint_response(response)


ARTICLE


In [14]:
where_filter = {
    "operator": "Or",
    "operands": [
        {
            "operator": "Equal",
            "path": ["keywords"],
            "valueText": "bonds",
        },
        {
            "operator": "Equal",
            "path": ["keywords"],
            "valueText": "fast money",
        },
    ],
}

response = client.query.get(class_name, properties).with_near_text(nearText).with_where(where_filter).with_limit(5).do()

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.6724188}
keywords: ['cnbc', 'Articles', 'World Markets', 'Bonds', 'Markets', 'Corporate bonds', 'Investment strategy', 'Corporate Debt', 'Investing', 'Credit and Debt', 'Squawk on the Street', 'Market Outlook', 'Europe Economy', 'source:tagname:CNBC US Source']
title: Lasry: Europe could face Japan-style deflation era
Response # 1
_additional: {'certainty': 0.65679705}
keywords: ['cnbc', 'Articles', 'CNBC TV', 'Fast Money Halftime Report', 'source:tagname:CNBC US Source']
title: EU, Iran to Impact Crude Oil Prices, Experts Say
Response # 2
_additional: {'certainty': 0.6142397}
keywords: ['cnbc', 'Articles', 'S&P 500 Index', 'Bank of America Corp', 'BHP Group Ltd', 'Freeport-McMoRan Inc', 'HP Inc', 'Intel Corp', 'Oracle Corp', 'US Bancorp', 'CBOE Volatility Index', 'United States Steel Corp', 'Industrial Select Sector SPDR Fund', "McDonald's Corp", 'CNBC TV', 'Fast Money', 'source:tagname:CNBC US Source']
title: Sellers Worried about Eur

## 1.4. Filter by minimum description length

In [15]:
nearText = {
    "concepts": ["car industry", "risks", "europe"],
    "certainty": 0.5,
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "keywords",
            "description",
            "_additional {certainty}",
        ],
    )
    .with_where(
        {
            "operator": "LessThan",
            "path": ["descriptionWordCount"],
            "valueInt": 50,
        }
    )
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.6416493}
description: More from CNBC.comEuropean Futures Rise after Fed CutLatest European Business News
keywords: ['cnbc', 'Articles', 'Europe News', 'Business News', 'Economy', 'World Economy', 'source:tagname:CNBC US Source']
title: Opening Calls for Europe's Major Indexes
Response # 1
_additional: {'certainty': 0.6053186}
description: Squawk Box Live in Europe kept you up to date with China's stock markets, which staged a late rally Wednesday. This came as investors geared up for another bumper day of earnings and the conclusion of this month's Federal Reserve monetary policy meeting.(App users please click here).
keywords: ['cnbc', 'Articles', 'FTSE 100', 'Stock markets', 'World Markets', 'DAX', 'CAC 40 Index', 'Europe News', 'Politics', 'stocks', 'Europe Markets', 'Europe Economy', 'CNBC TV', 'Video and TV', 'Europe Business Day', 'Squawk Box Europe', 'Squawk Box Live', 'source:tagname:CNBC Europe Source']
title: As it happened: C

## 1.5. Filter by title (text search)

In [16]:
nearText = {
    "concepts": ["market prediction"],
    "certainty": 0.5,
}

where_filter = {
    "operator": "Equal",
    "path": ["title"],
    "valueText": ["google motorola"],
}

response = (
    client.query.get(
        "Article",
        [
            "title",
            "description",
            "_additional {certainty}",
        ],
    )
    .with_where(where_filter)
    .with_near_text(nearText)
    .with_limit(5)
    .do()
)

pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.556478}
description: Google's $12.5 billion acquisition of Motorola Mobilitygives the Internet search company patent protection while putting it squarely into the smartphone hardware business, Executive Chairman Eric Schmidt told CNBC Monday. But the acquisition won't affect how Google  does business with the other phone makers that use its Android operating system, including HTC, he added."We are going into the hardware business, but we’re going to keep it separate and we’re going to treat everybody else on a fair basis," he explained.Android has had more than 550,000 activations a day and is succeeding "precisely because there are so many partners," he said. "If we somehow p--- them off ... it would be a disaster to the Android strategy."Google is also taking advantage of Motorola Mobility's  17,000+ patents, a portfolio Schmidt called "second to none" at a time when Google faces "significant patent challenges from people who want to 

## 1.6. "Like" filter (fuzzy match)

<u>**TODO: add link to this mentioning**</u>


Notes

Each query using the Like operator iterates over the entire inverted index for that property. The search time will go up linearly with the dataset size. Be aware that there might be a point where this query is too expensive and will not work anymore.

In [17]:
where_filter = {
    "operator": "Like",
    "path": ["name"],
    "valueString": "Katie H*",
}

response = (
    client.query.get(
        "Author",
        [
            "name",
        ],
    )
    .with_where(where_filter)
    .with_limit(5)
    .do()
)

pprint_response(response)


AUTHOR
Response # 0
name: Katie Holliday


# 2. Filter that accepts only raw string (GraphQL syntax)

## 2.1. Author name as a property

In [18]:
query = """
{
  Get {
    Article(where: {
        path: ["keywords"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "bonds"         # value (which is always = to the type of the path property)
      }, limit: 5) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
}
"""

response = client.query.raw(query)
pprint_response(response)


ARTICLE
Response # 0
hasAuthors: [{'name': 'Tom DiChristopher'}]
title: Instead of shunning Saudi Arabia after Khashoggi killing, investors flock to $7.5 billion bond sale
Response # 1
hasAuthors: [{'name': 'Silvia Amaro'}]
title: US Treasurys lower as investors focus on data, monitor Russia-US relations
Response # 2
hasAuthors: [{'name': 'Unknown'}]
title: 5 Stocks Insiders Love Right Now
Response # 3
hasAuthors: [{'name': 'Elliot Smith'}]
title: 10-year Treasury yield falls to 0.8% as investors return to safety amid pause in stock rally
Response # 4
hasAuthors: [{'name': 'Herb Greenberg'}]
title: Greenberg: Momentum King’s Next Big Move


In [19]:
query = """
{
  Get {
    Article(where: {
        path: ["{placeholder}"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "{placeholder}"         # value (which is always = to the type of the path property)
      }, limit: 5) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
}
"""


def format_raw_request(query: str, *args, template: str = "{placeholder}"):
    for arg in args:
        query = re.sub(template, arg, query, count=1)
    return query


query = format_raw_request(query, "keywords", "bonds")
print(query)

response = client.query.raw(query)
pprint_response(response)



{
  Get {
    Article(where: {
        path: ["keywords"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "bonds"         # value (which is always = to the type of the path property)
      }, limit: 5) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
}

ARTICLE
Response # 0
hasAuthors: [{'name': 'Tom DiChristopher'}]
title: Instead of shunning Saudi Arabia after Khashoggi killing, investors flock to $7.5 billion bond sale
Response # 1
hasAuthors: [{'name': 'Silvia Amaro'}]
title: US Treasurys lower as investors focus on data, monitor Russia-US relations
Response # 2
hasAuthors: [{'name': 'Unknown'}]
title: 5 Stocks Insiders Love Right Now
Response # 3
hasAuthors: [{'name': 'Elliot Smith'}]
title: 10-year Treasury yield falls to 0.8% as investors return to safety amid pause in stock rally
Response # 4
hasAuthors: [{'name': 'Herb Greenberg'}]
title: Greenberg: Momentum King’s Next B

In [20]:
query = """
{{
  Get {{
    Article(where: {{
        path: ["{path}"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "{valueText}"         # value (which is always = to the type of the path property)
      }}, limit: 5) {{
      title
      hasAuthors {{
        ... on Author {{
          name
        }}
      }}
    }}
  }}
}}
"""

query = query.format(path="keywords", valueText="bonds")
print(query)

response = client.query.raw(query)
pprint_response(response)



{
  Get {
    Article(where: {
        path: ["keywords"],    # Path to the property that should be used
        operator: Equal,  # operator
        valueText: "bonds"         # value (which is always = to the type of the path property)
      }, limit: 5) {
      title
      hasAuthors {
        ... on Author {
          name
        }
      }
    }
  }
}

ARTICLE
Response # 0
hasAuthors: [{'name': 'Tom DiChristopher'}]
title: Instead of shunning Saudi Arabia after Khashoggi killing, investors flock to $7.5 billion bond sale
Response # 1
hasAuthors: [{'name': 'Silvia Amaro'}]
title: US Treasurys lower as investors focus on data, monitor Russia-US relations
Response # 2
hasAuthors: [{'name': 'Unknown'}]
title: 5 Stocks Insiders Love Right Now
Response # 3
hasAuthors: [{'name': 'Elliot Smith'}]
title: 10-year Treasury yield falls to 0.8% as investors return to safety amid pause in stock rally
Response # 4
hasAuthors: [{'name': 'Herb Greenberg'}]
title: Greenberg: Momentum King’s Next B

## 2.2. Filter authors by number of wrote articles

In [21]:
# Find all articles that are written by authors that wrote at least two articles

query = """
  {{
    Get {{
      Author(
        where:{{
          valueInt: {limit},
          operator: GreaterThanEqual,
          path: ["wroteArticles"]
        }}, limit: 5
      ) {{
        name
        wroteArticles {{
          ... on Article {{
            title
          }}
        }}
      }}
    }}
  }}
"""

query = query.format(limit=2)
response = client.query.raw(query)
pprint_response(response)


AUTHOR
Response # 0
name: Leslie Josephs
wroteArticles: [{'title': "Georgia's lieutenant governor says he will 'kill' Delta tax break unless airline reinstates relationship with NRA"}, {'title': 'Raytheon and United Technologies agree to all-stock merger that would create aerospace behemoth'}]
Response # 1
name: Tyler Clifford
wroteArticles: [{'title': 'Cramer adds new stocks, recommends buying 12 laggards in his Covid-19 index'}, {'title': "'I come to bury Bitcoin, not to praise it': UBS\xa0"}]
Response # 2
name: Sharon Epperson
wroteArticles: [{'title': "Energy Falls Despite 'Above Average' Hurricane Forecast"}, {'title': "Want to start a business? Here's what you need to know"}]
Response # 3
name: Pippa Stevens
wroteArticles: [{'title': 'Guggenheim says solar sell-off is a buying opportunity and has an unusual favorite stock'}, {'title': 'Should CEO pay be tied to share price?'}]
Response # 4
name: Chris Morris
wroteArticles: [{'title': 'Video Game Sales Get off to Slow Start in 201

## 2.3. Filter articles by author

In [22]:
query = """
{{
  Get {{
    Article(
      where:{{
        valueString: "{authorName}",
        operator: Equal,
        path: ["hasAuthors", "Author", "name"]
      }}, limit: 5
    ) {{
      title
      hasAuthors {{
        ... on Author {{
          name
        }}
      }}
    }}
  }}
 }}
"""

query = query.format(authorName="Tyler")
response = client.query.raw(query)
pprint_response(response)


ARTICLE
Response # 0
hasAuthors: [{'name': 'Tyler Clifford'}]
title: Cramer adds new stocks, recommends buying 12 laggards in his Covid-19 index
Response # 1
hasAuthors: [{'name': 'Tyler Clifford'}]
title: 'I come to bury Bitcoin, not to praise it': UBS 
Response # 2
hasAuthors: [{'name': 'Tyler Bailey'}]
title: Your first trade for Wednesday, January 15


# 3. All combined

In [23]:
query = """
{
  Get {
    Article(
      limit: 5
      nearText: 
      {
        concepts: ["covid affect on stock market in usa"],
        moveAwayFrom:
        {
          concepts: ["new york", "cuomo"],
          force: 0.1
        }
      }
      where: 
      {
        operator: And,
        operands: 
        [
          {
            operator: GreaterThan,
            path: ["published_at"],
            valueDate: "2015-01-01T0:00:00.52Z",
          },
          {
            operator: Equal,
            path: ["keywords"],
            valueText: "bonds",
          },
          {
            operator: Equal,
            path: ["title"],
            valueText: "Europe Japan",
          },
          {
            operator: Equal,
            path: ["hasAuthors", "Author", "name"],
            valueString: "Tom DiChristopher",
          },
          {
            operator: GreaterThan,
            path: ["descriptionWordCount"],
            valueInt: 100,
          }
        ]
      }
    ) 
    {
      title
      description
      keywords
      published_at
      descriptionWordCount
      hasAuthors {
        ... on Author {
          name
        }
      }
      _additional {
        certainty
      }
    }
  }
}
"""

response = client.query.raw(query)
pprint_response(response)


ARTICLE
Response # 0
_additional: {'certainty': 0.6688515}
description: Europe could be looking at a Japan-style deflationary environment  for the next five years, investor Marc Lasry told CNBC on  Wednesday. Read MoreEuro tests low last  seen at its birth in1999  Lasry's Avenue Capital is continuing to buy credit-side debt at a  discount in Europe. Over the last three or four years, the amount  of debt that European banks have sold has increased by 100  percent, he said in a "Squawk  Box" interview.   "The way that the banks were able to sell this debt is, they keep  on buying sovereign debt, and then through that they make their  profits, and then each year they end up using those profits to  offset losses on that. And that's sort of of what happened in  Japan over a 10 year period," said Lasry, who specializes in  distressed debt investments.  Read More Can cash-strapped Europe  prevent terrorism surge?  The way for Europe to avoid a deflationary period is to clock 4  to 5 percent G